In [3]:
! pip install -U boto3

In [9]:
import logging
import time

import boto3
from botocore.client import BaseClient

logger = logging.getLogger(__name__)

In [27]:
class BedrockKBAgent():
    def __init__(self, region_name="us-east-1"):
        
        self.region_name = region_name

    def _return_aws_service_client(self, resource_name='bedrock', run_time=True) -> BaseClient:

        if resource_name == "bedrock":
            if run_time:
                service_client = boto3.client(
                    service_name="bedrock-agent-runtime",
                    region_name=self.region_name)
            else:
                service_client = boto3.client(
                    service_name="bedrock-agent",
                    region_name=self.region_name)
        elif resource_name == "iam":
            service_client = boto3.resource("iam")
            
        return service_client

    def retrieve_from_kb(self,kb_id,query):
        client = self._return_aws_service_client(run_time=True)
        response = client.retrieve_and_generate(
            input={
                    'text': query
            },
            retrieveAndGenerateConfiguration={
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-v2:1',
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                   #'numberOfResults': 123,
                    'overrideSearchType': 'HYBRID'
                                             }
                                        } 
                                     },
               'type': 'KNOWLEDGE_BASE'
                           } 
            )
        #print(response)
        return response

In [33]:
if __name__ == "__main__":
    kb = BedrockKBAgent()
    query="""
<text> 안에 있는 것은 쇼핑몰에서 파는 상품의 구매자 리뷰야. 
  <text>
  이번에 신행 갔다가 돌아 오는 길에 향수를 샀습니다.
  집에서 언박싱 해봤는데 향이 아주 좋고 꿀템같아요.  다른분들께도  강추 합니다!
  </text>
일본 사람이 자연스럽게 이해 할 수 있도록 번역해줘
           """
    response = kb.retrieve_from_kb("QJSPIIKOGD",query)
    #print(response)
    print(response['output']['text'])

この度は新婚旅行で香水をお買い上げいただき、ありがとうございます。お家に帰って開封したところ、香りがとても良く、大満足の一品でしたね。他の方にもぜひおすすめしたいです!
